# Overview

This notebook imports raw ws3 input data, reformats and monkey-patches the data, and exports Woodstock formatted input data files (which we will use in other DSS notebooks for this case as the input data files). 

# Set up environment

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import ws3.forest, ws3.core
import math
import csv
import numpy as np
from util import schedule_harvest_areacontrol

Define some key model parameters (will get used but defined here up top for convenience).

In [ ]:
period_length = 10
max_age =  1000

# Import and reformat inventory and yield input data

Read forest inventory data into memory (vector polygon GIS data layer with attribute table, in ESRI Shapefile format). This dataset represents a small subset of timber supply area (TSA) 17 in British Columbia. We monkey-patch the inventory data here to make it line up nicely with what we need downstream as input for the ws3 model (i.e., changes we make here to the in-memory dataset are not saved to the original dataset on disk). Most of what we are doing here is setting up the _theme_ columns in the attribute table, which should help newer ws3 users make the connection between input data and the landscape themes in ws3 model further down.

In [ ]:
stands = gpd.read_file('data/tsa17_subset/stands.shp')
stands = stands.rename(columns={'thlb':'theme1', 'au':'theme2', 'ldspp':'theme3', 'age2015':'age', 'shape_area':'area' })
stands['area'] = stands.geometry.area * 0.0001 # monkey-patch broken area attribute
stands.insert(4, 'theme4', stands['theme2'])
stands['theme2'] = stands['theme2'].astype(int)

Read yield data from a CSV file and recast AU column data type to integer.

In [ ]:
yld = pd.read_csv('data/yld.csv')
yld['AU'] = yld['AU'].astype(int)

Create analysis unit (AU) dataframe from stands dataframe data.

In [ ]:
AU = pd.DataFrame(stands['theme2']).drop_duplicates()
AU.rename(columns={'theme2':'AU'}, inplace=True)

Join `AU` and `yld` dataframes.

In [ ]:
yldmerged = pd.merge(AU, yld, on=['AU'], how='inner')

Import CANFI tree species lookup table (associates tree species names with integer numerical values, which we use as theme data values in the ws3 model), and insert species code values into the yield curve dataframe.

In [ ]:
canf = pd.read_csv('data/canfi_species_modified.csv')
canf = canf[['name', 'canfi_species']].set_index('name')

Burn CANFI species codes into stand and yield data tables.

In [ ]:
stands['theme3'] = stands.apply(lambda row: canf.loc[row['theme3'], 'canfi_species'], axis=1) 
yldmerged['canfi_species'] = yldmerged.apply(lambda row: canf.loc[row['LDSPP'], 'canfi_species'], axis=1)

Add a new `curve_id` colume that has same data values as `AU` column.

In [ ]:
yldmerged['curve_id'] = yldmerged['AU'] 

Save reformatted data to CSV files. 

In [ ]:
yldmerged.to_csv('data/yldmerged.csv', header=True, index=False)
stands.to_csv('data/stands_table.csv', header=True, index=False)

Rename stuff to match variable names we expect further down.

In [ ]:
stands_table = stands
curve_points_table = yldmerged
curve_points_table.set_index('AU', inplace=True)

# Export Woodstock-formatted input files 

We can use the new ws3 model instance we just built to export ws3 input files in Woodstock file format. We do this for three reasons. 

The first reason is that it will be simpler and more compact in the actual DSS notebook to instantiate the `ForestModel` object from these Woodstock-formatted files (and also this will provide an opportunity to demonstrate the existance and usage of the Woodstock model import functions that are built into ws3). 

The second reason is that the process of exporting data from a live `ws3.forest.ForestModel` instance to Woodstock-formatted input data files provides some insight into the internal structure and workings of ws3 models (which can be a challenging thing to get started with, particularly if you do not have a lot of experience building and running forest estate models). 

The third reason is that Woodstock file format is designed to be "human readable" (sort of... nobody ever said it would be super easy or super fun). Picking through the exported Woodstock-formatted files might help some people better understand the structure and details of the model we have built. If you have no experience reading Woodstock-formatted model input data files, then this is going to be trickier (unless you pause here and go take an introductory Woodstock training course of sort). Many forest professionals already have familiarity with Woodstock software and its special file format (through having been exposed to this at some point in their career). 

Start by creating a new subdirectory to hold the new Woodstock-formatted data files.

In [ ]:
!mkdir data/woodstock_model_files

## LANDSCAPE section

The LANDSCAPE section defines stratification variables (themes) and stratification variable values (basecodes). 

In [ ]:
theme_cols=['theme0', # TSA 
            'theme1', # THLB
            'theme2', # AU
            'theme3', # leading species code
            'theme4'  # yield curve ID
           ]
basecodes = [list(map(lambda x: str(x), stands_table[tc].unique())) for tc in theme_cols]
basecodes[2] = list(set(basecodes[2] + list(stands_table['theme2'].astype(str))))
basecodes[3] = list(set(basecodes[3] + list(stands_table['theme3'].astype(str))))
basecodes[4] = list(set(basecodes[4] + list(stands_table['theme4'].astype(str))))

In [ ]:
with open('data/woodstock_model_files/tsa17.lan', 'w') as file:
    print('*THEME Timber Supply Area (TSA)', file=file)
    print('tsa17',file=file)
    print('*THEME Timber Harvesting Land Base (THLB)', file=file)
    for basecode in basecodes[1]: print(basecode, file=file)
    print('*THEME Analysis Unit (AU)', file=file)
    for basecode in basecodes[2]: print(basecode, file=file)
    print('*THEME Leading tree species (CANFI species code)', file=file)
    for basecode in basecodes[3]: print(basecode, file=file)
    print('*THEME Yield curve ID', file=file)
    for basecode in basecodes[4]: print(basecode, file=file)

## AREAS section

The AREAS section defines the initial forest inventory, in terms of how many hectares of which age class are present in which development type (where a development type is defined as a unique sequence of landscape theme variable values).

In [ ]:
gstands = stands_table.groupby(theme_cols+['age'])

In [ ]:
with open('data/woodstock_model_files/tsa17.are', 'w') as file:
    for name, group in gstands:
        dtk, age, area = tuple(map(lambda x: str(x), name[:-1])), int(name[-1]), group['area'].sum()
        print('*A', ' '.join(v for v in dtk), age, area, file=file)

## YIELDS section

The YIELDS section defines yield curves (in this example we only track merchantable log volume, but we can use yield curves to track all sorts of other stuff). 

The following codes insert allomertic parameters to curve points table for biomass calculation purposes.

In [ ]:
coeff= pd.read_csv('data/allometic_parameters.csv')
coeff.set_index('canfi_species', inplace=True)

In [ ]:
coeff1= pd.read_csv('data/allometric_coefficient.csv')
coeff1.set_index('canfi_species', inplace=True)

Inserting the allometric parameters mentioned in P.Boudewyn et al. (2007)

In [ ]:
curve_points_table['a1'] = curve_points_table.apply(lambda row: coeff.loc[row['canfi_species'], 'a1'], axis=1)
curve_points_table['a2'] = curve_points_table.apply(lambda row: coeff.loc[row['canfi_species'], 'a2'], axis=1)
curve_points_table['a3'] = curve_points_table.apply(lambda row: coeff.loc[row['canfi_species'], 'a3'], axis=1)
curve_points_table['b1'] = curve_points_table.apply(lambda row: coeff.loc[row['canfi_species'], 'b1'], axis=1)
curve_points_table['b2'] = curve_points_table.apply(lambda row: coeff.loc[row['canfi_species'], 'b2'], axis=1)
curve_points_table['b3'] = curve_points_table.apply(lambda row: coeff.loc[row['canfi_species'], 'b3'], axis=1)
curve_points_table['c1'] = curve_points_table.apply(lambda row: coeff.loc[row['canfi_species'], 'c1'], axis=1)
curve_points_table['c2'] = curve_points_table.apply(lambda row: coeff.loc[row['canfi_species'], 'c2'], axis=1)
curve_points_table['c3'] = curve_points_table.apply(lambda row: coeff.loc[row['canfi_species'], 'c3'], axis=1)

In [ ]:
curve_points_table['a'] = curve_points_table.apply(lambda row: coeff1.loc[row['canfi_species'], 'a'], axis=1)
curve_points_table['b'] = curve_points_table.apply(lambda row: coeff1.loc[row['canfi_species'], 'b'], axis=1)


The following chunk of code extrapolates yield curve for biomass using allometric equations proposed by P.Boudewyn et al. (2007)

In [ ]:
with open('data/woodstock_model_files/tsa17.yld', 'w') as file:
        tot=[]
        tot_st=[]
        tot_bk=[]
        tot_br=[]
        tot_fl=[]
        swd=[]
        hwd=[]
        for AU, au_row in curve_points_table.iterrows():
            yname = 's%04d' % int(au_row.canfi_species)
            yname_st = 'st%04d' % int(au_row.canfi_species)
            yname_bk = 'bk%04d' % int(au_row.canfi_species)
            yname_br = 'br%04d' % int(au_row.canfi_species)
            yname_fl = 'fl%04d' % int(au_row.canfi_species)
            curve_id = au_row.curve_id
            mask = ('?', '?', str(AU), '?', str(curve_id))
            points = [(x*10, au_row['X%i' % (x*10)]) for x in range(36)]
            c = ws3.core.Curve(yname, points=points, type='a', is_volume=True, xmax=max_age, period_length=period_length)
            print('*Y', ' '.join(v for v in mask), file=file)
            print(yname, '1', ' '.join(str(int(c[x])) for x in range(0, 350, 10)), file=file)
            stemwood_points = [(x, au_row['a'] * (y ** au_row['b'])) for x, y in points]
            stemwood_curve = ws3.core.Curve(yname_st, points=stemwood_points, type='a', is_volume=True, xmax=max_age, period_length=period_length)
            print(yname_st, '1', ' '.join(str(int(stemwood_curve[x])) for x in range(0, 350, 10)), file=file)
            bark_points = [(x, math.exp(au_row['a1'] + au_row['a2'] * y + au_row['a3'] * math.log(y+5)) * au_row['a'] * (y ** au_row['b'])) for x, y in points]
            bark_curve = ws3.core.Curve(yname_bk, points=bark_points, type='a', is_volume=True, xmax=max_age, period_length=period_length)
            print(yname_bk, '1', ' '.join(str(round(bark_curve[x],3)) for x in range(0, 350, 10)), file=file)
            branches_points = [(x, math.exp(au_row['b1'] + au_row['b2'] * y + au_row['b3'] * math.log(y+5)) * au_row['a'] * (y ** au_row['b'])) for x, y in points]
            branches_curve = ws3.core.Curve(yname_br, points=branches_points, type='a', is_volume=True, xmax=max_age, period_length=period_length)
            print(yname_br, '1', ' '.join(str(round(branches_curve[x],3)) for x in range(0, 350, 10)), file=file)
            foliage_points = [(x, math.exp(au_row['c1'] + au_row['c2'] * y + au_row['c3'] * math.log(y+5)) * au_row['a'] * (y ** au_row['b'])) for x, y in points]
            foliage_curve = ws3.core.Curve(yname_fl, points=foliage_points, type='a', is_volume=True, xmax=max_age, period_length=period_length)
            print(yname_fl, '1', ' '.join(str(round(foliage_curve[x],3)) for x in range(0, 350, 10)), file=file)
            if yname_st not in tot_st:
                tot_st.append(yname_st)
            if yname_bk not in tot_bk:
                tot_bk.append(yname_bk)
            if yname_br not in tot_br:
                tot_br.append(yname_br)
            if yname_fl not in tot_fl:
                tot_fl.append(yname_fl)
            if yname not in tot:
                tot.append(yname)
            if int(au_row.canfi_species) > 1200:
                if yname not in hwd: hwd.append(yname)
            else:
                if yname not in swd: swd.append(yname)
        print('*YC ? ? ? ? ?', file=file)
        print('totvol _SUM(%s)' % ', '.join(map(str, tot)), file=file)
        print('totvol_st _SUM(%s)' % ', '.join(map(str, tot_st)), file=file)
        print('totvol_bk _SUM(%s)' % ', '.join(map(str, tot_bk)), file=file)
        print('totvol_br _SUM(%s)' % ', '.join(map(str, tot_br)), file=file)
        print('totvol_fl _SUM(%s)' % ', '.join(map(str, tot_fl)), file=file)
        print('swdvol _SUM(%s)' % ', '.join(map(str, swd)), file=file)
        print('hwdvol _SUM(%s)' % ', '.join(map(str, hwd)), file=file)

## ACTIONS section

The ACTIONS section defines actions that can be applied in the model (e.g., harvesting, planting, thinning, fertilization, etc). 

In [ ]:
with open('data/woodstock_model_files/tsa17.act', 'w') as file:
    print('ACTIONS', file=file)
    print('*ACTION harvest Y', file=file)
    print('*OPERABLE harvest', file=file)
    print('? 1 ? ? ? _AGE >= 90 AND _AGE <= 600', file=file)

## TRANSITIONS section

The TRANSITIONS section defines transitions (i.e., transition to a new development type and age class induced by applying a specific action to a specific combination of development type and age class). If there were no transitions in a forest estate model, it would simply be aging (i.e., growing) the forest forward from time step 1 through to time step N.

In [ ]:
with open('data/woodstock_model_files/tsa17.trn', 'w') as file:
    acode = 'harvest'
    print('*CASE', acode, file=file)
    record_au = set()
    for au_id, au_row in stands_table.iterrows():
        if au_row.theme2 in record_au: continue
        if not au_row.theme1: continue
        target_curve_id = au_row.theme4  
        smask = ' '.join(('?', '?' , str(target_curve_id), '?', '?'))
        tmask = ' '.join(('?', '?' , '?', '?', str(target_curve_id)))
        print('*SOURCE', smask, file=file)
        print('*TARGET', tmask, '100', file=file)
        record_au.add(au_row.theme2)